In [1]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import logging
import time
import sys

PROJECT_PATH = "/Users/shawn/Documents/personal/rsi_divergence_detector"
sys.path.append(PROJECT_PATH)
# Load the training data
# Replace 'training_data.csv' with your actual data file or DataFrame
df_total = pd.read_pickle(f'{PROJECT_PATH}/data/processed_data/training_data.pickle')
divergence_data = pd.read_pickle(f"{PROJECT_PATH}/data/processed_data/divergence_data.pickle")

In [2]:
# Test dataframes sliced
start_time = "2024-11-01"
df = df_total[df_total.timeframe == '15m']
df = df.loc[df.index >= start_time]
# df = df.loc[df.index <= '2024-11-27 00:00:00']
# df = df.loc[df.index >= '2024-11-22 12:00:00']
# df = df.loc[df.index <= '2024-11-22 16:15:00']


dd_filter = divergence_data.copy()
for key, value in dd_filter.items():
    value = value.loc[value.index >= start_time]
    value = value.sort_values('end_datetime').sort_index()
    dd_filter[key] = value
    print(f"{key} filtered data length = {len(value)}")


5m filtered data length = 351
15m filtered data length = 133
1h filtered data length = 39
4h filtered data length = 10
1d filtered data length = 4


* dd_filter = divergence data 관련 data frame (분봉 별로 TP, SL, entry price 등이 나와있음)
* df = 가격 정보가 나와있음

## Adding different timeline divergence data

In [7]:
dd_filter['15m']

,end_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,TP,SL,label,...,SL_percent,position,TP_vs_SL,TP_/_SL,profit,div_5m,div_15m,div_1h,div_4h,div_1d
start_datetime,,,,,,,,,,,,,,,,,,,,,
2024-11-01 11:15:00,2024-11-01 12:45:00,2024-11-01 13:15:00,70202.83,2024-11-01 08:45:00,Bearish Divergence,997.14,12.20,70024.45,70467.65,False,...,0.38,False,0.673590,0.67,-264.82,False,True,False,False,False
2024-11-02 14:30:00,2024-11-03 01:00:00,2024-11-03 01:30:00,69118.68,2024-11-02 07:00:00,Bullish Divergence,-878.89,-18.95,69353.57,69100.01,False,...,0.03,True,12.581358,12.58,-18.67,False,True,False,False,False
2024-11-03 02:30:00,2024-11-04 21:30:00,2024-11-04 22:00:00,67120.56,2024-11-02 16:45:00,Bullish Divergence,908.01,24.32,67940.79,66905.30,True,...,0.32,False,3.810403,3.81,820.23,True,True,True,False,False
2024-11-03 02:30:00,2024-11-04 18:45:00,2024-11-04 19:15:00,68020.00,2024-11-02 16:45:00,Bullish Divergence,32.19,3.91,68394.83,67640.00,False,...,0.56,False,0.986400,0.99,-380.00,True,True,True,False,False
2024-11-03 02:30:00,2024-11-04 18:00:00,2024-11-04 18:30:00,67784.02,2024-11-02 16:45:00,Bullish Divergence,289.87,10.98,68330.54,67535.96,False,...,0.37,False,2.203158,2.20,-248.06,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-04 20:30:00,2024-12-05 04:45:00,2024-12-05 05:15:00,103108.00,2024-12-04 16:45:00,Bearish Divergence,-1433.56,-19.00,100001.05,103347.08,False,...,0.23,True,12.995455,13.00,-239.08,False,True,False,False,False
2024-12-05 17:00:00,2024-12-05 18:45:00,2024-12-05 19:15:00,100560.00,2024-12-05 14:45:00,Bullish Divergence,-1595.29,-7.20,101462.81,100172.67,False,...,0.39,False,2.330855,2.33,-387.33,False,True,False,False,False
2024-12-06 18:15:00,2024-12-06 20:30:00,2024-12-06 21:00:00,101360.00,2024-12-06 12:45:00,Bearish Divergence,-1459.08,-22.72,100223.26,101898.99,True,...,0.53,False,2.109020,2.11,1136.74,False,True,False,False,False
